In [7]:
import tensorflow as tf
import numpy as np
width = 224
height = 224

data_X_train = np.load("data/X_train.npy", allow_pickle=True)
data_y_train = np.load("data/y_train.npy", allow_pickle=True)
data_labels_train = np.load("data/labels_train.npy", allow_pickle=True)
#data_X_test = np.load("data/X_test.npy")
#data_y_test = np.load("data/y_test.npy")
print(data_X_train.shape)
print(data_y_train.shape)
print(data_labels_train.shape)


(33402, 128, 128, 3)
(33402, 6, 4)
(33402, 6)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train_bboxes, y_val_bboxes, y_train_labels, y_val_labels = train_test_split(
    data_X_train, data_y_train, data_labels_train,
    test_size=0.2,
    random_state=5,
    shuffle=True
)
data_X_train = None
data_y_train = None
data_labels_train = None


In [9]:
ds_train = tf.data.Dataset.from_tensor_slices((X_train, {"bboxes": y_train_bboxes, "classes": y_train_labels}))
X_train = None
y_train_bboxes = None
y_train_labels = None
ds_val = tf.data.Dataset.from_tensor_slices((X_val, {"bboxes": y_val_bboxes, "classes": y_val_labels}))
X_val = None
y_val_bboxes = None
y_val_labels = None
ds_train = ds_train.shuffle(buffer_size=10000).batch(128).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.batch(128).prefetch(tf.data.AUTOTUNE)

In [19]:

# TODO
def predModel(input_shape=(128,128,3), num_classes = 11, max_digits = 6):
    inputs = tf.keras.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(64, (3,3), activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D((2,2))(x)
    x = tf.keras.layers.Conv2D(128, (3,3), activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    #BBOX
    bbox = tf.keras.layers.Dense(256, activation='relu')(x)
    bbox = tf.keras.layers.Dense(max_digits * 4)(bbox)
    bbox_output = tf.keras.layers.Reshape((max_digits, 4), name="bboxes")(bbox)

    #CLASSIFICATION
    cls = tf.keras.layers.Dense(256, activation='relu')(x)
    cls = tf.keras.layers.Dense(max_digits * num_classes)(cls)
    cls_output = tf.keras.layers.Reshape((max_digits, num_classes), name="classes")(cls)

    return tf.keras.Model(inputs, [bbox_output, cls_output])

model = predModel()
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss={
        "bboxes": "mse",
        "classes": "sparse_categorical_crossentropy"
    },
    metrics={
        "bboxes": ["mae"],
        "classes": ["accuracy"]
    }
#    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)


model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_val,
)




Epoch 1/6
209/209 ━━━━━━━━━━━━━━━━━━━━ 102s 482ms/step - bboxes_loss: 0.0081 - bboxes_mae: 0.0455 - classes_accuracy: 0.5553 - classes_loss: 2.4323 - loss: 2.4405 - val_bboxes_loss: 0.0071 - val_bboxes_mae: 0.0430 - val_classes_accuracy: 0.5307 - val_classes_loss: 2.2589 - val_loss: 2.2633
Epoch 2/6
209/209 ━━━━━━━━━━━━━━━━━━━━ 111s 533ms/step - bboxes_loss: 0.0071 - bboxes_mae: 0.0425 - classes_accuracy: 0.5554 - classes_loss: 1.4758 - loss: 1.4832 - val_bboxes_loss: 0.0072 - val_bboxes_mae: 0.0425 - val_classes_accuracy: 0.5611 - val_classes_loss: 1.4053 - val_loss: 1.4148
Epoch 3/6
209/209 ━━━━━━━━━━━━━━━━━━━━ 152s 728ms/step - bboxes_loss: 0.0071 - bboxes_mae: 0.0421 - classes_accuracy: 0.5643 - classes_loss: 1.4051 - loss: 1.4121 - val_bboxes_loss: 0.0071 - val_bboxes_mae: 0.0418 - val_classes_accuracy: 0.6554 - val_classes_loss: 1.4218 - val_loss: 1.4311
Epoch 4/6
209/209 ━━━━━━━━━━━━━━━━━━━━ 134s 643ms/step - bboxes_loss: 0.0071 - bboxes_mae: 0.0423 - classes_accuracy: 0.5017 - 

In [20]:
results = model.evaluate(ds_val)
print(results)
print('Test loss: %.4f accuracy: %.4f' % (results[2], results[4]))

53/53 ━━━━━━━━━━━━━━━━━━━━ 7s 127ms/step - bboxes_loss: 0.0068 - bboxes_mae: 0.0410 - classes_accuracy: 0.4093 - classes_loss: 1.6213 - loss: 1.6288
[1.6288394927978516, 0.006828800309449434, 1.6212964057922363, 0.04096950218081474, 0.40929508209228516]
Test loss: 1.6213 accuracy: 0.4093


In [ ]:
'''
# Build a simple neural network model
model = tf.keras.Sequential([
#   tf.keras.Input(shape=(32, 32, 3)),
    tf.keras.layers.Resizing(48, 48, pad_to_aspect_ratio = True, fill_mode='constant', fill_value=0.0),
#    tf.keras.layers.Rescaling(scale = 1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    #tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
X_train = np.transpose(data_X_train, (3,0,1,2))
y_train = data_y_train.reshape(-1)
X_test = np.transpose(data_X_test, (3,0,1,2))
y_test = data_y_test.reshape(-1)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))
ds_train = ds_train.shuffle(buffer_size=10000).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

# Train and validate

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)
'''